In [8]:
import os
import warnings
import pickle    # chunk, vectorDB 저장한것 사용
from dotenv import load_dotenv

# 경고메세지 삭제
warnings.filterwarnings('ignore')
load_dotenv()

# openapi key 확인
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    raise ValueError('.env확인,  key없음')

# 필수 라이브러리 로드
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
import time

class SimpleRAGSystem:
    '''간단한 RAG 시스템 래퍼 클래스'''
    def __init__(self, vectorstore, llm, retriever_k=3):
        self.vectorstore = vectorstore
        self.llm = llm
        self.retriever = vectorstore.as_retriever(search_kwargs={'k':retriever_k})
        self.chain = self._build_chain()
    
    
    def _build_chain(self):
        '''RAG 체인 구성'''
        prompt = ChatPromptTemplate.from_messages([
            ('system', "당신은 제공된 문맥을 바탕으로 질문에 답변하는 AI입니다.\
            문맥에 없는 정보는 답하지 마세요"),
            ('human', '문맥:\n{context}\n\n질문:{question}\n\n답변:')
        ])
        return(
            {'context': self.retriever | self._format_docs, 'question':RunnablePassthrough()}
            | prompt
            | self.llm
            | StrOutputParser()
        )
    
    
    @staticmethod   # 클래스 안에 있지만, 그 클래스의 인스턴스(self)도, 클래스(cls)도 필요 없이 쓸 수 있는 함수를 만들 때 사용하는 데코레이터
    def _format_docs(docs):
        return '\n\n'.join([doc.page_content for doc in docs])
    
    def ask(self, question:str) -> str:
        '''질문에 답변'''
        return self.chain.invoke(question)
    
    def ask_with_sources(self, question:str) -> dict : 
        '''질문에 답변 + 출처 반환'''
        answer = self.chain.invoke(question)
        sources = self.retriever.invoke(question)
        return {
            'answer' : answer,
            'source' : [ doc.metadata.get('source', 'unknown') for doc in sources]
        }
    


if __name__ == '__main__' :
    embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')
    # 이전단계에서 저장한 vectordb로드
    persist_dir = './chroma_db_reg2'
    vectorstore = Chroma(
        persist_directory = persist_dir,
        collection_name = 'persistent_rag',
        embedding_function = OpenAIEmbeddings(model = 'text-embedding-3-small')
    )
    llm = ChatOpenAI( model = 'gpt-4o-mini', temperature=0 )

    rag_system = SimpleRAGSystem(vectorstore, llm)

    print('래퍼 클래스 테스트: ')
    result = rag_system.ask_with_sources("VectorDB의 종류를 알려주세요")
    print(f'질문: VectorDB의 종류를 알려주세요')
    print(f"답변: {result['answer']}")  #일부만 보고 싶으면 ['answer'][:100]
    print(f"출처: {result['source']}")

래퍼 클래스 테스트: 
질문: VectorDB의 종류를 알려주세요
답변: 주요 VectorDB 솔루션은 다음과 같습니다:

- ChromaDB: 로컬 개발에 적합한 오픈소스 솔루션으로, 파이썬 네이티브로 설치가 간편합니다.
- Pinecone: 완전 관리형 클라우드 서비스로, 대규모 프로덕션 환경에 적합합니다.
- Weaviate: 그래프 기반 벡터 데이터베이스로, 하이브리드 검색을 지원합니다.
출처: ['vectordb_intro.txt', 'vectordb_intro.txt', 'rag_concept.txt']


In [6]:
items = vectorstore._collection.get()
metas = items['metadatas']
contents = items['documents']

for i, (meta, content) in enumerate(zip(metas, contents)):
    print(f"\n---- 문서 {i+1} ----")
    print("source:", meta.get("source", "없음"))
    print("page:", meta.get("page", "없음"))
    print("내용:", content[:200], "...")



---- 문서 1 ----
source: langchain_intro.txt
page: 없음
내용: LangChain은 대규모 언어 모델(LLM)을 활용한 애플리케이션 개발을 위한 프레임워크입니다.

        LangChain의 주요 구성 요소:
        1. Models: 다양한 LLM 제공자(OpenAI, Anthropic, Google 등)와 통합
        2. Prompts: 프롬프트 템플릿 관리 및 최적화
        3. Ch ...

---- 문서 2 ----
source: langchain_intro.txt
page: 없음
내용: 4. Memory: 대화 맥락을 유지하기 위한 메모리 시스템
        5. Indexes: 문서 검색을 위한 인덱싱 도구
        6. Agents: 도구를 사용하여 복잡한 작업을 수행하는 에이전트

        LangChain Expression Language (LCEL)은 체인을 구성하는 선언적 방식으로,
        파이프(|) 연산 ...

---- 문서 3 ----
source: rag_concept.txt
page: 없음
내용: RAG (Retrieval-Augmented Generation)는 검색 증강 생성 기술입니다.

        RAG의 작동 원리:
        1. 사용자 질문을 임베딩 벡터로 변환합니다.
        2. 벡터 데이터베이스에서 유사한 문서를 검색합니다.
        3. 검색된 문서를 컨텍스트로 사용하여 LLM이 답변을 생성합니다.

       ...

---- 문서 4 ----
source: rag_concept.txt
page: 없음
내용: - 환각(Hallucination)을 감소시킵니다. 실제 문서 기반으로 답변하기 때문입니다.
        - 출처를 명시할 수 있습니다. 어떤 문서에서 정보를 가져왔는지 추적 가능합니다.
        - 도메인 특화가 가능합니다. 특정 분야의 문서만 사용하여 전문적인 답변을 제공합니다.

 